In [1]:
import json
import re
import spacy as sp
import pandas as pd

with open('data/all_situations.json') as f:
    situations = json.load(f)

print(f'There are {len(situations)} situations in the dataset')

There are 1574 situations in the dataset


## Exploratory Data Analysis

#### Create dataframe with useful information

In [2]:
situation_data = {
    'related_subject': [situation['subject'].get('id', None) if situation['subject'] else "" for situation in situations],
    'id': [situation['id'] for situation in situations],
    'situation_title': [situation['title'] for situation in situations],
    'url': [situation['link'] for situation in situations],
    #'content': [situation['content'] for situation in situations],
    'icpc': [', '.join(situation['codes']['icpc']) if situation['codes']['icpc'] else "" for situation in situations],
    #'medical_source': [', '.join(situation['codes']['medical_source']) if situation['codes']['medical_source'] else None for situation in situations],
    #'related_situations': [', '.join(related_situation['title'] for related_situation in situation['related']['situations']) if situation['related']['situations'] else None for situation in situations]
}

situation_df = pd.DataFrame(situation_data)

situation_df.to_csv('data/situation_data.csv', index=False)

In [ ]:
#give me all the situations that have icpc code F75


In [18]:
task_data = {
    'id': [situation['id'] for situation in situations],
    'situation_title': [situation['title'] for situation in situations],
    'content_text': ['\n'.join([content_item.get('value', '') for item in situation.get('content', []) for content_item in item.get('item', [])]) for situation in situations],
}

task_df = pd.DataFrame(task_data)
task_df

task_df.to_csv('data/task_data.csv', index=False)


In [17]:
for i in range(5):
    values = '\n'.join([entry['item'][0]['value'] for entry in situation_df['content'][i]])
    print(values)

<ul><li>De oogarts onderzoekt uw ogen met kleurstof en blauw licht.</li>
	<li>Er zijn verschillende behandelingen:
	<ul><li>oogzalf met aciclovir</li>
		<li>druppels met medicijnen tegen ontsteking</li>
		<li>tabletten met medicijnen die virussen remmen</li>
	</ul></li>
	<li>Meestal geneest de ontsteking zonder blijvende schade.</li>
</ul>
<p>Het herpesvirus kan een oogontsteking veroorzaken. Verschillende delen van het oog kunnen ontstoken raken:</p><ul><li>Het bindvlies. Dat is het doorzichtige slijmvlies dat het oogwit en de binnenkant van de oogleden bedekt.</li><li>De oogleden.</li><li>Het hoornvlies. Dat is het doorzichtige laagje voor de iris en de pupil.</li></ul>
<p>De oogarts onderzoekt uw oog. U krijgt  een oranje kleurstof in uw oog. De oogarts bekijkt uw oog daarna met blauw licht. De oranje kleurstof wordt dan geel. Door de kleurstof kan de oogarts blaasjes en beschadigingen duidelijk zien.</p>

<p>Heeft u veel pijn aan uw oog? Dan kan de oogarts uw oog eerst verdoven met

#### Exploration of ICPC codes

In [7]:
situations_no_icpc = []
situations_icpc = []
count = 0
for i in range(len(situation_df)):
    situation_id = situation_df['id'][i]
    situation_title = situation_df['situation_title'][i]
    icpc_code = situation_df['icpc'][i]

    if icpc_code is None:
        situations_no_icpc.append((situation_id, situation_title))
    else:
        # Check for invalid ICPC codes and correct them
        if ';' in icpc_code:
            count+= 1
            icpc_code = icpc_code.replace(';', ',')
            situation_df.loc[situation_df['id'] == situation_id, 'icpc'] = icpc_code

        if 'so' in icpc_code:
            icpc_code = icpc_code.replace('so', 's0')

        situations_icpc.append((situation_id, situation_title, icpc_code))

print(f'Number of situations with no ICPC codes: {len(situations_no_icpc)}')
print(f'Number of situations with ICPC codes: {len(situations_icpc)}')

Number of situations with no ICPC codes: 258
Number of situations with ICPC codes: 1316


In [8]:
situation_multiple_icpc = [item for item in situations_icpc if ',' in item[2]]
print(f'Number of situations with multiple ICPC codes: {len(situation_multiple_icpc)}')

Number of situations with multiple ICPC codes: 401


In [9]:
situation_specific_icpc = [item for item in situations_icpc if '.' in item[2]]
print(f'Number of situations with specific ICPC codes: {len(situation_specific_icpc)}')

Number of situations with specific ICPC codes: 78


In [10]:
situation_df[situation_df['situation_title'].str.contains('buikpijn', case=False)]

,id,situation_title,related_subject,content,icpc,medical_source,related_situations
305,15035,Mijn kind gaat naar de kinderarts voor buikpij...,Buikpijn bij kinderen,"[{'index': 0, 'type': 'text', 'category': 'In ...",D01,FMS,"Mijn kind heeft buikpijn, Ik wil iets doen aan..."
385,11808,Mijn kind heeft buikpijn die lang duurt,Buikpijn bij kinderen,"[{'index': 0, 'type': 'text', 'category': 'In ...",D01,NHG,"Mijn kind heeft buikpijn, Ik wil iets doen aan..."
583,11674,Ik heb buikpijn,Buikpijn,"[{'index': 0, 'type': 'text', 'category': 'In ...",None,NHG,Ik heb bloedverlies en/of buikpijn in de eerst...
901,11421,Ik heb bloedverlies en/of buikpijn in de eerst...,Zwanger,"[{'index': 0, 'type': 'text', 'category': 'In ...","W03, W82",NHG,"Ik ben zwanger: belangrijke adviezen, Ik ben z..."
951,11807,Ik wil iets doen aan de buikpijn bij mijn kind,Buikpijn bij kinderen,"[{'index': 0, 'type': 'text', 'category': 'In ...","D01, D06",NHG,"Mijn kind heeft buikpijn, Mijn kind heeft buik..."
952,11805,Mijn kind heeft buikpijn,Buikpijn bij kinderen,"[{'index': 0, 'type': 'text', 'category': 'In ...","D01, D06",NHG,Ik wil iets doen aan de buikpijn bij mijn kind...
953,11834,Ik ga een buikpijndagboek bijhouden,Buikpijn bij kinderen,"[{'index': 0, 'type': 'text', 'category': 'In ...","D01, D06",NHG,"Mijn kind heeft buikpijn, Ik wil iets doen aan..."
1553,20009,Ik krijg een behandeling voor buikpijn door ee...,Zenuwen klem in buikwand,"[{'index': 0, 'type': 'text', 'category': 'In ...",D06,FMS,Ik heb steeds buikpijn door een zenuw die klem...
1554,19989,Ik heb steeds buikpijn door een zenuw die klem...,Zenuwen klem in buikwand,"[{'index': 0, 'type': 'text', 'category': 'In ...",None,FMS,Ik krijg een behandeling voor buikpijn door ee...


In [11]:
#ICPC codes can be used to classify the situations into different categories. 
#The categories are: symptomen en klachten (01-29), diagnostische/preventieve verrichtingen (30-49), 
#medicatie/therapeutische verrichtingen (50-59), uitslagen van onderzoek (60-61), administratieve verrichtingen (62), v
#erwijzingen/andere verrichtingen (63-69), ziekten (70-99).

situation_df['icpc_category'] = situation_df['id'].apply(lambda x: [])

def assign_icpc_category(category, situation_id):
    categories_map = {
        (1, 29): 'symptomen en klachten',
        (30, 49): 'diagnostische/preventieve verrichtingen',
        (50, 59): 'medicatie/therapeutische verrichtingen',
        (60, 61): 'uitslagen van onderzoek',
        (62, 62): 'administratieve verrichtingen',
        (63, 69): 'verwijzingen/andere verrichtingen',
        (70, 99): 'ziekten'
    }

    categories = []
    for cat_range, cat_name in categories_map.items():
        if cat_range[0] <= category <= cat_range[1]:
            categories.append(cat_name)

    return categories

for situation in situations_icpc:
    icpc_code = situation[2].replace(" ", "")
    categories = set()

    if ',' in icpc_code:
        categories_list = icpc_code.split(',')
        for category in categories_list:
            #if not empty
            if category:
                category = int(category[1:3])
                new_categories = assign_icpc_category(category, situation[0])
                for new_cat in new_categories:
                    categories.add(new_cat)
    else:
        category = int(icpc_code[1:3])
        new_categories = assign_icpc_category(category, situation[0])
        for new_cat in new_categories:
            categories.add(new_cat)

    # Find the indices where the condition is true
    indices = situation_df.index[situation_df['id'] == situation[0]].tolist()

    # Update categories for each index separately
    for idx in indices:
        current_categories = set(situation_df.at[idx, 'icpc_category'])
        current_categories.update(categories)
        situation_df.at[idx, 'icpc_category'] = list(current_categories)


In [9]:
icpc_category_counts = situation_df['icpc_category'].apply(pd.Series).stack().value_counts()
icpc_category_counts

/var/folders/8d/5kdw2bjs4h79gy4zy389_pmm0000gn/T/ipykernel_11128/1671784390.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  icpc_category_counts = situation_df['icpc_category'].apply(pd.Series).stack().value_counts()


ziekten                                    1007
symptomen en klachten                       509
diagnostische/preventieve verrichtingen      17
medicatie/therapeutische verrichtingen        3
dtype: int64

In [10]:
situation_df[(situation_df['medical_source'].str.contains('FMS', na=False)) & (situation_df['medical_source'].str.contains('NHG', na=False) == False) & (situation_df['icpc'].notnull())]

#skip content en icpc_category
#situation_df.drop(columns=['content', 'icpc_category'], inplace=True)


,id,situation_title,related_subject,content,icpc,medical_source,related_situations,icpc_category
0,16574,Ik ga naar de oogarts voor een oogontsteking d...,Oogontsteking door herpes,"[{'index': 0, 'type': 'text', 'category': 'In ...",F73,FMS,"Ik heb een oogontsteking door herpes, De oogon...",[ziekten]
6,29130,Ik wil zorgen dat ik geen wondroos krijg,Wondroos,"[{'index': 0, 'type': 'text', 'category': 'In ...",S76,FMS,"Ik heb wondroos, Ik heb vaker wondroos",[ziekten]
15,28928,Ik heb eczeem aan mijn handen en ga naar de hu...,Eczeem,"[{'index': 0, 'type': 'text', 'category': 'In ...","S87, S88",FMS,"Ik heb eczeem, Ik heb eczeem door erfelijke aa...",[ziekten]
17,28538,Ik heb eczeem door contact met iets en ga naar...,Contacteczeem,"[{'index': 0, 'type': 'text', 'category': 'In ...",S88,FMS,Ik heb eczeem door contact met iets,[ziekten]
27,18136,Ik krijg een hersenscan bij onderzoek naar de ...,Ziekte van Parkinson,"[{'index': 0, 'type': 'text', 'category': 'In ...",N87,FMS,Ik ga naar het ziekenhuis voor onderzoek naar ...,[ziekten]
...,...,...,...,...,...,...,...,...
1564,19291,Ik denk na over kinderen krijgen. Heb ik kans ...,Zwanger worden en erfelijke ziektes,"[{'index': 0, 'type': 'text', 'category': 'In ...",A99,FMS,Ik wil een kind en er komt een ziekte voor in ...,[ziekten]
1566,17480,Ik heb klachten na een borstvergroting,Borstvergroting,"[{'index': 0, 'type': 'text', 'category': 'In ...",X29,FMS,"Ik wil misschien een borstvergroting, Ik krijg...",[symptomen en klachten]
1567,17246,Ik krijg een borstvergroting,Borstvergroting,"[{'index': 0, 'type': 'text', 'category': 'In ...",X29,FMS,"Ik wil misschien een borstvergroting, Ik heb k...",[symptomen en klachten]
1568,17492,Ik heb een borstvergroting en wil meedoen aan ...,Borstvergroting,"[{'index': 0, 'type': 'text', 'category': 'In ...",X29,FMS,"Ik wil misschien een borstvergroting, Ik krijg...",[symptomen en klachten]


#### Exploration of medical sources

In [11]:
situations_no_medical_source = []
for i in range(len(situation_df)):
    if situation_df['medical_source'][i] is None:
        situations_no_medical_source.append((situation_df['id'][i], situation_df['situation_title'][i]))

print(f'Number of situations with no medical source: {len(situations_no_medical_source)}')
situations_no_medical_source

Number of situations with no medical source: 28


[(12315, 'Hoe kan ik verbranding voorkomen?'),
 (11297, 'Ik ben uitgenodigd voor het bevolkingsonderzoek borstkanker'),
 (12233, 'Ik maak me zorgen om iemand met psychische klachten'),
 (11906, 'Ik word onderzocht op epilepsie'),
 (11905, 'Ik heb een aanval gehad'),
 (11907, 'Ik wil goed omgaan met mijn epilepsie'),
 (15165,
  'Ik ben zwanger en wil mijn baby beschermen tegen kinkhoest (22 wekenprik)'),
 (11268, 'Ik heb hoge bloeddruk'),
 (11824, 'Ik wil mijn botten sterk houden en voorkomen dat ik een bot breek'),
 (11956, 'Ik heb klachten door de eikenprocessierups'),
 (11257, 'Ik heb astma'),
 (11284, 'Ik gebruik medicijnen voor astma'),
 (19423,
  'Ik heb een uitnodiging gekregen voor mijn kind van 14 maanden voor prikken'),
 (14496, 'Ik ga medicijnen gebruiken bij een hoog cholesterol'),
 (11141, 'Ik geef over'),
 (12279, 'Ik heb een ontsteking van de huid rond mijn mond'),
 (12422, 'Ik heb een keratoacanthoom'),
 (21072, 'Ik heb een plekje op mijn huid en wil weten wat het is'),


In [12]:
for situation in situations_no_medical_source:
    meer_informatie = situation_df[situation_df['id'] == situation[0]]['content'].values[0][-1:]
    for item in meer_informatie:
        item_value = item['item'][0]['value']  
        if 'richtlijn' in item_value:  # Check if the word "richtlijn" is present in item_value
            print("The word 'richtlijn' is present in the value:", situation)
        else:
            print(f"The word 'richtlijn' is not present in the value:", situation)

The word 'richtlijn' is present in the value: (12315, 'Hoe kan ik verbranding voorkomen?')
The word 'richtlijn' is present in the value: (11297, 'Ik ben uitgenodigd voor het bevolkingsonderzoek borstkanker')
The word 'richtlijn' is not present in the value: (12233, 'Ik maak me zorgen om iemand met psychische klachten')
The word 'richtlijn' is not present in the value: (11906, 'Ik word onderzocht op epilepsie')
The word 'richtlijn' is not present in the value: (11905, 'Ik heb een aanval gehad')
The word 'richtlijn' is not present in the value: (11907, 'Ik wil goed omgaan met mijn epilepsie')
The word 'richtlijn' is present in the value: (15165, 'Ik ben zwanger en wil mijn baby beschermen tegen kinkhoest (22 wekenprik)')
The word 'richtlijn' is present in the value: (11268, 'Ik heb hoge bloeddruk')
The word 'richtlijn' is present in the value: (11824, 'Ik wil mijn botten sterk houden en voorkomen dat ik een bot breek')
The word 'richtlijn' is not present in the value: (11956, 'Ik heb kla

#### Exploration of overarching subjects

In [13]:
situations_no_subject = []
for i in range(len(situation_df)):
    if situation_df['related_subject'][i] is None:
        situations_no_subject.append((situation_df['id'][i], situation_df['situation_title'][i]))

print(f'Number of situations with no overarching subject: {len(situations_no_subject)}')
situations_no_subject

Number of situations with no overarching subject: 11


[(11329, 'Ik denk na over de griepprik'),
 (27397, 'Mijn partner heeft de ziekte van Parkinson'),
 (27605, 'Ik wil leren omgaan met de ziekte van Parkinson'),
 (27391, 'Ik ben bang dat ik de ziekte van Parkinson heb'),
 (22343, 'Ik word getest op griep'),
 (22357, 'Ik heb griep en ik krijg virusremmers'),
 (11328, 'Ik heb griep'),
 (12453, 'Ik kies voor een maagverkleining'),
 (12454, 'Ik heb een maagverkleining gehad'),
 (22367, 'Ik heb griep en ik word opgenomen in het ziekenhuis'),
 (18320, 'Ik denk na over een maagverkleining en ik wil graag een kind')]

#### Exploration of categories in content

In [14]:
category_labels = {}

for situation in situations:
    for content_item in situation.get('content', []):
        content_category = content_item.get('category')
        category_labels[content_category] = category_labels.get(content_category, 0) + 1

print("Category labels and their counts:")
for category, count in category_labels.items():
    print(f"{category}: {count}")

print(f'There are {len(category_labels)} category labels in the data.')

Category labels and their counts:
In het kort: 1573
Wat is het: 995
Onderzoeken: 291
Behandeling: 418
Adviezen: 733
Hoe gaat het verder: 990
Wanneer bellen: 833
Meer informatie: 1535
Hoe werkt plas ophouden: 2
Oorzaken: 596
Hulpmiddelen: 10
Medicijnen: 325
Zijn er medicijnen?: 1
Hoe doe ik het: 2
Plasdagboek: 1
Hulp bij blaastraining: 1
Plas ophouden: 1
Blaastraining: 2
Wat is het?: 25
Geen wondroos krijgen: 1
Wat kan helpen?: 1
Goed voor je huid zorgen: 1
Wondjes verzorgen: 1
Over deze tekst: 65
Film Wondroos: 2
Wat merk ik: 462
Wie heeft grotere kans: 1
Kan het kwaad: 116
Adviezen niet weer wondroos: 1
Wat kan ik zelf doen: 2
Film: 168
Besmetting: 4
Prik: 3
Wanneer: 18
Wanneer niet: 4
Voordelen en nadelen: 24
Griepprik: voor wie: 1
Film: Uitnodiging voor de griepprik: 1
Griepprik: wanneer: 1
Griepprik: werking: 1
Griepprik: elk jaar: 1
Griepprik: voordelen en nadelen: 1
Wat is griep: 1
Film: Griep voorkomen: 1
Besmetting voorkomen: 1
Prik tegen pneumokokken: 1
Zalven en crèmes: 1
Beh

#### Exploration of related situations

In [15]:
situations_no_related = []
for i in range(len(situation_df)):
    if situation_df['related_situations'][i] is None:
        situations_no_related.append((situation_df['id'][i], situation_df['situation_title'][i]))

print(f'Number of situations with no related situations: {len(situations_no_related)}')

situations_no_related

Number of situations with no related situations: 236


[(11601, 'Ik heb te veel haargroei'),
 (11530, 'Ik heb spierreuma (polymyalgia rheumatica)'),
 (16414, 'Ik denk erover om de prik tegen pneumokokken te halen'),
 (11550, 'Ik heb Q-koorts'),
 (11524, 'Ik heb de ziekte van Pfeiffer'),
 (28301, 'Mijn werk heeft een negatieve invloed op mijn privé-leven'),
 (11196, 'Ik heb een bijholteontsteking'),
 (11617, 'Ik heb last van een verstopt oor'),
 (11086, 'Ik heb een aangezichtsverlamming van Bell'),
 (15141, 'Ik krijg ineens dikke ogen of lippen'),
 (11817, 'Ik heb een golfarm'),
 (27851, 'Mijn baby zat tijdens de bevalling vast met de schouder'),
 (26528, 'Ik ga speciale oefeningen doen tegen duizeligheid'),
 (11117, 'Ik kies voor een hormoonring'),
 (11119, 'Ik kies voor een hormoonstaafje'),
 (11116, 'Ik kies voor een hormoonpleister om niet zwanger te worden'),
 (15999, 'Ik kies voor de prikpil'),
 (11642, 'Ik denk dat mijn kind mazelen heeft'),
 (28471,
  'Ik ben zwanger en veel te zwaar en wil me voorbereiden op de bevalling'),
 (28453

## Entity Linking

#### Preparing text (not necessary, since it is done in the EL module)

In [24]:
html_pattern = re.compile(r'<[^>]+>')
escape_pattern = re.compile(r'\\u[0-9a-fA-F]{4}')
newline_pattern = re.compile(r'(\n|\t|\xa0|\u202f)')

with open('data/situation_content.txt', 'w') as f:
    for situation in situations:
        f.write('Situatie: '+ situation.get('title', '') + ' (' + str(situation.get('id', ''))+')'+  '\n')
        for content_item in situation.get('content', []):
            item_value = content_item.get('item', [{}])[0].get('value', '')
            #cleaned_value = newline_pattern.sub(' ', escape_pattern.sub('', html_pattern.sub('', item_value)))
            f.write(item_value + '\n')
        f.write('\n')


nlp = sp.load('nl_core_news_sm')

situation_content = []
for situation in situations:
    content = ''
    for content_item in situation.get('content', []):
        item_value = content_item.get('item', [{}])[0].get('value', '')
        cleaned_value = newline_pattern.sub(' ', escape_pattern.sub('', html_pattern.sub('', item_value)))
        content += cleaned_value + ' '
    situation_content.append(content)

situation_content

situation_content[0]

doc = nlp(situation_content[0])
for token in doc:
    print(token.text, token.pos_, token.dep_)

De DET det
oogarts NOUN nsubj
onderzoekt VERB ROOT
uw PRON nmod:poss
ogen NOUN obj
met ADP case
kleurstof NOUN obl
en CCONJ cc
blauw ADJ amod
licht NOUN conj
. PUNCT punct
  SPACE _sp
Er ADV advmod
zijn VERB ROOT
verschillende ADJ amod
behandelingen NOUN nsubj
: PUNCT punct
  SPACE _sp
oogzalf ADJ amod
met ADP case
aciclovir NOUN nmod
   SPACE _sp
druppels NOUN nsubj
met ADP case
medicijnen NOUN nmod
tegen ADP case
ontsteking NOUN nmod
   SPACE _sp
tabletten NOUN nmod
met ADP case
medicijnen NOUN nmod
die DET det
virussen NOUN nsubj
remmen VERB parataxis
    SPACE _sp
Meestal ADV advmod
geneest VERB parataxis
de DET det
ontsteking NOUN obj
zonder ADP case
blijvende VERB amod
schade NOUN obl
. PUNCT punct
  SPACE _sp
Het DET det
herpesvirus NOUN nsubj
kan AUX aux
een DET det
oogontsteking NOUN obj
veroorzaken VERB ROOT
. PUNCT punct
Verschillende ADJ amod
delen NOUN nsubj
van ADP case
het DET det
oog NOUN nmod
kunnen AUX aux
ontstoken VERB ROOT
raken VERB xcomp
: PUNCT punct
Het DET det